In [58]:
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import barnard_exact, ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [59]:
def load_data(data_paths):
    df = pd.DataFrame()
    for p in data_paths:
        print(f'loading {p}')
        _df = pd.read_excel(p)
        df = pd.concat([df, _df])
    
    display(df.head())
    display(df.info())
    return df

data_paths = [r'..\S1 - No-dig\data.xlsx', r'..\S3 - No-dig fennel\data.xlsx']
df = load_data(data_paths)

loading ..\S1 - No-dig\data.xlsx
loading ..\S3 - No-dig fennel\data.xlsx


,plant_id,bed_id,bed_type,distance_to_east_wall_mm,distance_to_south_wall_mm,sow_date,transplant_date,survived_to_harvest,height_a_1,height_b_1,...,height_3,height_date_3,harvest_date,yield_g,wall_distance_e_mm,wall_distance_s_mm,species,strain,height_30d_mm,height_30d_date
0,1,A,no-dig,NaN,NaN,2024-01-15,2024-02-02,1,70.0,69.0,...,230.0,2024-05-04,NaT,NaN,2000,3800,Brassica aleracea var Botrytis,Candid Charm F1 Hybrid,NaN,NaT
1,2,B,dig,NaN,NaN,2024-01-15,2024-02-02,1,70.0,77.0,...,270.0,2024-05-04,2024-07-24,197.4,2000,3200,Brassica aleracea var Botrytis,Candid Charm F1 Hybrid,NaN,NaT
2,3,A,no-dig,NaN,NaN,2024-01-15,2024-02-02,0,NaN,NaN,...,NaN,NaT,NaT,NaN,2500,3800,Brassica aleracea var Botrytis,Candid Charm F1 Hybrid,NaN,NaT
3,4,B,dig,NaN,NaN,2024-01-15,2024-02-02,1,75.0,75.0,...,270.0,2024-05-04,2024-07-28,270.2,2500,3200,Brassica aleracea var Botrytis,Candid Charm F1 Hybrid,NaN,NaT
4,5,C,dig,NaN,NaN,2024-01-15,2024-02-02,1,131.0,135.0,...,410.0,2024-05-04,2024-06-17,431.8,3000,3800,Brassica aleracea var Botrytis,Candid Charm F1 Hybrid,NaN,NaT


<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 0 to 17
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   plant_id                   30 non-null     int64         
 1   bed_id                     30 non-null     object        
 2   bed_type                   30 non-null     object        
 3   distance_to_east_wall_mm   0 non-null      float64       
 4   distance_to_south_wall_mm  0 non-null      float64       
 5   sow_date                   12 non-null     datetime64[ns]
 6   transplant_date            12 non-null     datetime64[ns]
 7   survived_to_harvest        30 non-null     int64         
 8   height_a_1                 9 non-null      float64       
 9   height_b_1                 9 non-null      float64       
 10  height_date_1              9 non-null      datetime64[ns]
 11  height_a_2                 9 non-null      float64       
 12  height_b_2     

None

In [60]:
def transform_data(df):
    print(f'resetting_index')
    df.reset_index(inplace=True, drop=True)

    print('making dummy columns')
    df = pd.merge(df, pd.get_dummies(df['bed_type'], drop_first=True, dtype=float), left_index=True, right_index=True)
    df = pd.merge(df, pd.get_dummies(df['species'], drop_first=True, dtype=float), left_index=True, right_index=True)

    return df

df = transform_data(df)
df.info()

resetting_index
making dummy columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   plant_id                   30 non-null     int64         
 1   bed_id                     30 non-null     object        
 2   bed_type                   30 non-null     object        
 3   distance_to_east_wall_mm   0 non-null      float64       
 4   distance_to_south_wall_mm  0 non-null      float64       
 5   sow_date                   12 non-null     datetime64[ns]
 6   transplant_date            12 non-null     datetime64[ns]
 7   survived_to_harvest        30 non-null     int64         
 8   height_a_1                 9 non-null      float64       
 9   height_b_1                 9 non-null      float64       
 10  height_date_1              9 non-null      datetime64[ns]
 11  height_a_2                 9 non-nul

# Yield

In [61]:
def drop_na(df, col):
    print(f'There are {len(df)} samples.')
    mask = df[col].isna()
    print(f'Excluding {sum(mask)} samples with nan {col}')
    df = df[mask==False]
    print(f'{len(df)} samples remain.')
    return df

def yield_linear_regression(df):
    df = drop_na(df, 'yield_g')
    # X = np.array(df[['no-dig', 'wall_distance_e_mm', 'wall_distance_s_mm', 'species']])
    # y = np.array(df[['yield_g']])
    # X = sm.add_constant(X)

    # mod = sm.OLS(y, X)
    mod = smf.ols('yield_g ~ bed_type + wall_distance_e_mm + wall_distance_s_mm + species', data=df)
    res = mod.fit()

    print(res.summary())
    print_odds_ratio(res)

    return [mod, res]
    
mod, res = yield_linear_regression(df)


There are 30 samples.
Excluding 11 samples with nan yield_g
19 samples remain.
                            OLS Regression Results                            
Dep. Variable:                yield_g   R-squared:                       0.630
Model:                            OLS   Adj. R-squared:                  0.524
Method:                 Least Squares   F-statistic:                     5.955
Date:                Sun, 28 Jul 2024   Prob (F-statistic):            0.00515
Time:                        20:54:45   Log-Likelihood:                -108.14
No. Observations:                  19   AIC:                             226.3
Df Residuals:                      14   BIC:                             231.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

c:\Users\acbre\anaconda3\envs\default\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
c:\Users\acbre\anaconda3\envs\default\Lib\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: overflow encountered in exp
  result = func(self.values, **kwargs)


# Survival

In [62]:
def print_odds_ratio(res):
    odds_ratios = pd.DataFrame(
        {
            "OR": res.params,
            "Lower CI": res.conf_int()[0],
            "Upper CI": res.conf_int()[1],
        }
    )
    odds_ratios = np.exp(odds_ratios)

    print('Odds ratios')
    print(odds_ratios)

def survival_logistic_regression(df):
    df = drop_na(df, 'survived_to_harvest')

    mod = smf.logit('survived_to_harvest ~ bed_type + wall_distance_e_mm + wall_distance_s_mm + species', data=df)
    res = mod.fit()

    print(res.summary())
    print_odds_ratio(res)

    return [mod, res]
    
mod, res = survival_logistic_regression(df)

There are 30 samples.
Excluding 0 samples with nan survived_to_harvest
30 samples remain.
Optimization terminated successfully.
         Current function value: 0.516745
         Iterations 7
                            Logit Regression Results                           
Dep. Variable:     survived_to_harvest   No. Observations:                   30
Model:                           Logit   Df Residuals:                       25
Method:                            MLE   Df Model:                            4
Date:                 Sun, 28 Jul 2024   Pseudo R-squ.:                  0.1089
Time:                         20:54:46   Log-Likelihood:                -15.502
converged:                        True   LL-Null:                       -17.397
Covariance Type:             nonrobust   LLR p-value:                    0.4351
                                    coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------